In [2]:
import pandas as pd

In [3]:
# load data
stockCodes = ["AMZN", "APPL", "META", "NFLX"]
stocks_df = pd.DataFrame()

for stockCode in stockCodes:
    df = pd.read_csv(f"Resources/{stockCode}-HistoricalData.csv")
    df = df.rename(columns={'Close/Last': 'Close'})
    df['StockCode'] = stockCode
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    df['Close'] = df['Close'].str.replace('$', '')
    df['Open'] = df['Open'].str.replace('$', '')
    df['High'] = df['High'].str.replace('$', '')
    df['Low'] = df['Low'].str.replace('$', '')
    df = df.convert_dtypes()
    #display(df)
    stocks_df = pd.concat([stocks_df,df])

display(stocks_df.dtypes)
display(stocks_df)

# save the CSV in one file
stocks_df.to_csv(f"Resources/clean-HisotricalData.csv",mode = 'w', index=False)

Date         string[python]
Close        string[python]
Volume                Int64
Open         string[python]
High         string[python]
Low          string[python]
StockCode    string[python]
dtype: object

,Date,Close,Volume,Open,High,Low,StockCode
0,2024-03-04,177.58,37381520,177.53,180.14,177.49,AMZN
1,2024-03-01,178.22,31981150,176.75,178.725,176.07,AMZN
2,2024-02-29,176.76,53805360,173.01,177.22,172.85,AMZN
3,2024-02-28,173.16,28180480,172.44,174.05,172.27,AMZN
4,2024-02-27,173.54,31141730,174.075,174.62,172.86,AMZN
...,...,...,...,...,...,...,...
2512,2014-03-11,62.4971,10447489,63.2343,63.8114,62.2357,NFLX
2513,2014-03-10,62.85,15433892,64.0671,64.1108,61.7287,NFLX
2514,2014-03-07,64.0528,11570639,64.8671,64.9343,63.6443,NFLX
2515,2014-03-06,64.3586,11803362,64.9914,65.4286,64.1411,NFLX
